# Homework 3 - What is the best anime in the world?
--- 

Word provided by :
 - Valentino Sacco 1949934
 - Stephanie Tahtouh 1944339
 - Camilla Savarese 1637288
 - Anthony Giusti 1992108

NB. To facilitate notebook comprehension, we've extracted all functions into three main files:
 - `./scraping_utils.py`
 - `./indexing_utils.py`
 - `./query_utils.py`

All functions have been *documented*, so if you have questions about what they do, just *hover with your mouse on the function*!

*Used libraries:*

In [2]:
# Install BeautifulSoup, this will be needed to crawl the web
#!pip3 install beautifulsoup4
#!pip3 install tqdm
#!pip3 install nltk
#!pip3 install tabulate

*Notebook Imports*

In [4]:
# Import asyncio, this will be needed to perform asynchronous operations
import os
import asyncio
# Importing multiprocessing to assign operat+ions to threadpools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
# Importing this to create necessary directories
import pathlib
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import nltk
from joblib import Parallel, delayed
import json 
import math
from collections import Counter
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None


nltk.download('stopwords')
nltk.download('punkt')


import scraping_utils
import indexing_utils
import query_utils

'''if this returns an error on tabulate, install it by running the cell above.'''

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Valentino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'if this returns an error on tabulate, install it by running the cell above.'

Since many processes in this homework took a lot of computational time, we've decided to do most of the work using **multiprocessing techniques**.   
here is defined our global thread pool:

In [5]:
'''
Defining the amount of cores available for the process to use. If this slows your machine too much, hardcode it. 
EX: AVAILABLE_CORES = 4
'''
AVAILABLE_CORES = multiprocessing.cpu_count()
print(AVAILABLE_CORES)

# Initializing ThreadPools 
pool = ThreadPool(AVAILABLE_CORES)

12


## Data Collection 
---

### 1.1. Get the list of animes   

Here we collect urls for the requested 400 pages of the top animes section.   

In order to do so, we first fetch the first 400 pages and store them into an array,   
In particular we use   
```pool.map(lambda page : scraping_utils.fetch_urls_in_page(page, pages), pages)```   
to assign the process to many threads

then, for each page we scrape all anime URLs and save them in a txt file but also store them in that very same array.

In [5]:
# Defining pages variables based on how many pages we want to retrieve
pages = [None] * 400
pages_num = range(0,400)

# Crawl Top Animes pages
pool.map(lambda num : scraping_utils.fetch_page(num, pages), pages_num)   

print("Done fetching the pages!")
print("Going to fetch urls")
# Scraping all URLs present in the crawled pages
pool.map(lambda page : scraping_utils.fetch_urls_in_page(page, pages), pages)

print("Done fetching urls")

100%|████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<?, ?it/s]

Done fetching the pages!
Going to fetch urls


Done fetching urls


After fetching all the URLs we need, we proceed to save them in a TXT file, which can be found under:   

*./dataset/anime_urls.txt*

In [15]:
saved_file = scraping_utils.save_urls_to_txt(pages)
if(saved_file):
    print("Saved all URLs in ./dataset/anime_urls.txt")
else :
    print("Couldn't save URLs in txt!")

written 19139 lines
Saved all URLs in ./dataset/anime_urls.txt


## 1.2. Crawl animes

After obtaining all urls we proceed to fetch each url's html and save it in a file under its relative page directory.  
All html can be found under :
`./dataset/pages_*/`

In [ ]:
# Fetch animes for every requested page
'''
Here we fetch and save animes in html files. 
Starting_page defines from which page you want to resume the process. (It works as an index)

EX: 
    to start from scratch:
        starting_page = 0
    if you want to start from the 10th page:
        starting_page = 9
    if you want to set 200 as an upper bound:
        last_page = 199   
'''
starting_page = 384
last_page = len(pages)
pages_to_process = pages[starting_page:]
for i in range(0, len(pages_to_process)) : 

    '''
    This functions uses Multiprocessing to assign to given amount of threads the fetching
    and saving progress of all given urls into HTMLs.
    '''
    scraping_utils.fetch_animes_and_save_file(pages_to_process[i], starting_page+i+1, AVAILABLE_CORES)

Since every html page contains **metadatas** in order to retrieve the URL of each anime, instead of reading everytime the txt, we extract the url directly from the html page.

Here's an example on the output of the method:

In [3]:
with open('./dataset/page_1/article_1.html', 'r', encoding="utf-8") as f:
    url = scraping_utils.extract_url_from_html( f.read() )
    print(url)

https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood


## 1.3 Parse downloaded pages
---

The list of information we desire for each anime and their format is the following:

- Anime Name (to save as animeTitle): String.  
- Anime Type (to save as animeType): String.   
- Number of episode (to save as animeNumEpisode): Integer.   
- Release and End Dates of anime (to save as releaseDate and endDate): Convert both release and end date into datetime format.     
- Number of members (to save as animeNumMembers): Integer.    
- Score (to save as animeScore): Float.   
- Users (to save as animeUsers): Integer    
- Rank (to save as animeRank): Integer.    
- Popularity (to save as animePopularity): Integer.    
- Synopsis (to save as animeDescription): String.    
- Related Anime (to save as animeRelated): Extract all the related animes, but only keep unique       values and those that have a hyperlink associated to them. List of strings.        
- Characters (to save as animeCharacters): List of strings.         
- Voices (to save as animeVoices): List of strings.     
- Staff (to save as animeStaff): Include the staff  name and their responsibility/task in a list of lists.     

In [4]:
# find all files into directories
matches = pathlib.Path("./dataset").glob("**/*.html")
# multiprocess parsing every html into a tsv
result = Parallel(n_jobs=AVAILABLE_CORES)(delayed(scraping_utils.extract_informations_from_anime_html)(path) for path in tqdm(matches))
#pool.map(scraping_utils.extract_informations_from_anime_html, matches)

19124it [24:57, 12.77it/s]


Here we first instance a path generator that performs a lookup on the `./dataset` folder to find every html file.

Then, to speed up the process as possible, we make use of the Parrallel library, which allows us to divide into many parallel jobs the process of parsing every html into a tsv with the informations we are looking for.

With this code, computing more than 19,000 html files takes only 24 minutes.

To see how we parse every html into a tsv, you can find the function `extract_informations_from_anime_html` inside the `scraping_utils` module.

# 2. Search Engine   
---

To preprocess every tsv we computed, we made use of the nltk library as requested. 
In particular our steps of preprocessing are :

**Removing stopwords**   
**Removing punctuation**   
**Stemming**

It's important to note that our punctuation processing is custom and does not remove complex words like "full-metal" but it translates them into "fullmetal"

In [4]:
tsv_matches = pathlib.Path("./tsv_dataset").glob("*.tsv")
for match in tsv_matches:
    indexing_utils.preprocess_tsv(match)

Every preprocessed tsv gets parsed into a json for performance reasons and can be found under the path    
`./preprocessed_dataset/`

## 2.1. Conjunctive query   
---


### 2.1.1 Create your index!   

- Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

For performance reasons we've decided to use json as our vocabulary format. 
We've also decided to *multiprocess the hydration of the vocabulary* and therefore did not chosse to assign a sequential integer as our term id.
Instead, we've decided to use hashcoding as a reliable function to define every word's _id

In [19]:
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
indexing_utils.hydrate_vocabulary_from_files(preprocessed_files, AVAILABLE_CORES)

Our resulting vocabulary can be found under :   
`./vocabulary.json`

### Our first Inverted Index

Below we create our first inverted index.
Making use of the previously created vocabulary, we assign to every word in a document's Synopsis a record containing as value every document id of the documents containing that specific word.

The index can be found under `./indexes/synopsis_index.json`

N.B.: The function used below as been written both using multiprocessing and to be reused for the creation of the next inverted index. Feel free to read its documentation by hovering with your mouse of by looking inside `./indexing_utils`

In [4]:
inverted_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
indexing_utils.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(inverted_index_path, preprocessed_files, vocabulary_path, AVAILABLE_CORES)

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/synopsis_index.json


### 2.1.2) Execute the query

In order to execute the query we allow the user to provide us a string!
To see how the query is evaluated, you can find the code inside `./query_utils.py`   
Don't worry about typos, if you misspell a word, the function will tell you to check it!

In [7]:
query_utils.take_simple_query_from_user()

Hi! Type in your query : 
 saiyan race
Results for "saiyan race" : 4 | elapsed time: 0.1091 seconds
| Title                                                   | Description   | Url                                                                                       |
|---------------------------------------------------------+---------------+-------------------------------------------------------------------------------------------|
| Dragon Ball Super: Broly                                | Forty-on..    | https://myanimelist.net/anime/36946/Dragon_Ball_Super__Broly                              |
| Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen | Bardock,..    | https://myanimelist.net/anime/986/Dragon_Ball_Z_Special_1__Tatta_Hitori_no_Saishuu_Kessen |
| Dragon Ball Kai                                         | Five yea..    | https://myanimelist.net/anime/6033/Dragon_Ball_Kai                                        |
| Dragon Ball Z                                           | 

## 2.2.1) Our Second Inverted index

Here we create our second inverted index, considering the tfidf score of each word.    
Let's explain what a **tfidf** score is:   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$tfidf$ is the product of **$tf$**, which is the **term frequency factor of a word inside a document**:        
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$tf$ = (# of term occurences in the document) $/$ (# of words in the document)    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$idf$ = $log($(# of docs) $/$ (# of docs containing current word)$)$

This results in something like this:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
}
```

The index can be found under  `'./indexes/tf_idf_synopsis_index.json`

In [3]:
inverted_index_path = './indexes/tf_idf_synopsis_index.json'
idf_source_index_path = './indexes/synopsis_index.json'
vocabulary_path = './vocabulary.json'
Path("./indexes").mkdir(parents=True, exist_ok=True)
preprocessed_files = pathlib.Path("./preprocessed_dataset").glob("*.json")
tot_amount_of_documents = 19125
want_idf = True

'''
this function compiles the inverted based on this logic:

it uses the first index to find out in how many docs a word is included, 
it finds the _id of each word via the vocabulary and (since want_idf == True)
it computes the tfidf score of each word and saves it in a tuple like so :

term_id_2:[(document1, tfIdf_{term,document1})
'''
indexing_utils.hydrate_synopsys_inverted_index_with_given_files_and_vocabulary(
    inverted_index_path,
    preprocessed_files,
    vocabulary_path,
    AVAILABLE_CORES,
    want_idf,
    tot_amount_of_documents,
    idf_source_index_path,
    )

1------------starting hydrating inverted index-------------
1------------inverted index not yet existant---------------
1------------reading vocabulary----------------------------
1------------proceeding to multiprocess inputs-------------
1------------finished to multiprocess inputs---------------
1------------finished hydrating inverted index-------------
1------------dumping inverted index -----------------------
1------------dumped inverted index to path-----------------
./indexes/tf_idf_synopsis_index.json


Here we ask the user to provide us with query and the amount of documents he/she wants to be retrieved. 

This is how we evaluate the query :

First we compute the interception of words sets to find out what documents contain every word the user is looking for.   
Then we generate two kind of vectors, a query_vector and many documents_vectors :

- we fill the query vector components with 1 if the word corresponding to the component is present in the query, or 0 if it's not present.    
- We also fill the documents vectors components with the TFIDF of each term leaving 0 on the components of the words not contained in the doc.    
- Then we compute the similarity between the query vector and each one of the document vectors as the cosine:   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$cos(theta)$ = $(qd)/(|q||d|)$
- We then store the cosines in a heap structure, in order to make sorting more efficient.
- We retrieve the top k (in this case, we ask the user to provide $k$ ) scores, and associate them to the corresponding documents, in order to show the correct ranking in the final output.

In [9]:
query_utils.take_top_k_of_query_from_user()

Hi! Type in your query : 
 dragon ball super
How many results would you like to retrieve?
 5
Results for "dragon ball super" : 7 | elapsed time: 1.425398 seconds
| Title                                                        | Description   | Url                                                                                           |   Similarity |
|--------------------------------------------------------------+---------------+-----------------------------------------------------------------------------------------------+--------------|
| Dragon Ball Super: Super Hero                                | Sequel m..    | https://myanimelist.net/anime/48903/Dragon_Ball_Super__Super_Hero                             |     0.638038 |
| Dragon Ball Z: The Real 4-D at Super Tenkaichi Budokai       | Dragon B..    | https://myanimelist.net/anime/42449/Dragon_Ball_Z__The_Real_4-D_at_Super_Tenkaichi_Budokai    |     0.611956 |
| Dragon Ball Super                                            | Seven

# 3. Define a new score!
---

As our new score, we decided to implement the jaccard similarity algorithm :

The jaccard similarity index is computed like so:
\begin{equation*}
\frac {A \cap B}{A \cup B}
\end{equation*}

Where A is the set based on the query words and B is the set composed by the element we are analyzing. 
This formula is applied to every component of the documents, resulting in a vector having as components the Jaccard Indexes for every field: this allows us to have a score that instead of only relying on the synopsis, relies also on:

- animeTitle 
- animeDescription 
- animeRelated 
- animeCharacters 

Then, to better adapt the results to the user preferences, we have given a weight to each field of the documents that changes based on the preference inputs received by the user when asking for the query.

In the end, since the user is also asked how many results he wants to be retrieved, we compute a max-heap of the scores in order to retrieve the best $k$ k elements and return them to the user.

Every function that act in this process can be found inside the query module : `./query_utils.py` feel free to read the documentations to have a better understanding of the process.

#### Let's say that we are looking for an old dragon ball movie...

so we should look for *very few episodes*, a *quite old anime* with *kind of a good fanbase*, we also may be interested in a *good score* and the anime, since it's old, to *not be too popular...*

In [4]:
query_utils.take_biased_query_from_user()

Hi! Type in your query : 


 dragon ball


How many results would you like to retrieve?


 5


You will now be asked to type in some preferences. 
Consider the range from -5 to 5, where -5 is the least, 5 is the max and 0 is indifferent.

How many episodes do you prefer?


...

Are you looking for an anime with a good score?


 4


Are you looking for a very popular anime?


 2


+----+---------------------------------------------------------+--------------------+-------------------------------------------------------------------------------------------+--------------+
|    | animeTitle                                              | animeDescription   | url                                                                                       |   similarity |
|----+---------------------------------------------------------+--------------------+-------------------------------------------------------------------------------------------+--------------|
|  0 | Dragon Ball Z Movie 02: Kono Yo de Ichiban Tsuyoi Yatsu | In his l...        | https://myanimelist.net/anime/895/Dragon_Ball_Z_Movie_02__Kono_Yo_de_Ichiban_Tsuyoi_Yatsu |     1        |
|  0 | Dragon Ball GT                                          | Emperor ...        | https://myanimelist.net/anime/225/Dragon_Ball_GT                                          |     0.959212 |
|  0 | Dragon Ball Z: Summer Vacati

##  5 ALGORITHMC QUESTION

Basically, given a list of integer and positive numbers (they are appointment's durations),we have to find the maximum sum of subsequence where the subsequence contains no element at adjacent positions.
We approached the problem using dynamic programming.
We create an auxiliary list $L_1$.

$L_1[i]$ represents the maximum possible sum for the sub-array staring from index ‘i’ and ending at index ‘N-1’, and $V$ is the initial input of length N.

For every index ‘i’, we have two choices:

1) Choose the current index:
   So that we will have 
   $L_1[i]$ = $V[i] + L_1[i-2]$


2) Skip the current index:
   $L_1[i]$ = $L_1[i-1]$
   
We will choose the path that maximizes our result:  $L_1[i]$ = max(  $V[i] + L_1[i-2]$, $L_1[i-1]$ )



$L_1[N-1]$ is the longest possible duration.

- Observation 1: In this way the computational time is $O(N)$
- Observation 2: We have to take care of the trivial cases, $N=0,1,2$

This is the implemtented program:

In [ ]:
import numpy as np
n=int(input("How many appointment requests are there? --->"))
l = []
for i in range(1,n+1):
        x=int(input("Insert a value that correspond to the duration of the appointment  --->"))
        l.append(x)
        
print (l) #our list

How many appointment requests are there? --->6
Insert a value that correspond to the duration of the appointment  --->30
Insert a value that correspond to the duration of the appointment  --->40
Insert a value that correspond to the duration of the appointment  --->25
Insert a value that correspond to the duration of the appointment  --->50
Insert a value that correspond to the duration of the appointment  --->30
Insert a value that correspond to the duration of the appointment  --->20
[30, 40, 25, 50, 30, 20]


The function below is used to find the longest possible duration.

In [ ]:
def max_seq(l):
    n=len(l)
    
    #basic cases
    if n == 0:
        return 0
    if n == 1:
        return l[0]
    if n == 2:
        return max(l[0], l[1])

    l_1 = [0]*n #new list
    l_1[0] = l[0]
    l_1[1] = max(l[0], l[1])

    for i in range(2, n):
        l_1[i] = max(l[i]+l_1[i-2], l_1[i-1])

    print("The longest possible duration is :" )
    print(l_1[-1])

    return l_1

In [ ]:
max_seq(l)

The longest possible duration is :
110


[30, 40, 55, 90, 90, 110]

In this way we have found the maximum possible sum, but not the values that allow us to obtain it.
We initialize c with the maximum possible duration.
To extract the elements we  scroll through the auxiliary list starting from the end:
If the element in position i is greater than the next, it means that it is one of the elements that interest us.
So we append it in another auxiliary variable, and we subtract that value from our sum.
The boolean is needed to indicate the fact that if the element of index i has been taken from the list, the adjacent one must be skipped.


In [ ]:
def extract_sub(l,l_1):
    l_1= l_1[::-1] #invert
    l= l[::-1]
    c= l_1[0]
    sub= []
    boolean = True
    N=len(l_1)
    for i in range(N):
        
        if i == N-1 and c > 0:
            sub.append(l[i])
            c-= l[i]
        elif not boolean:
            boolean = True
            continue
        elif  i < N-1 and l_1[i] > l_1[i+1] and boolean:
            boolean = False
            sub.append(l[i])
            c-= l[i]
    print("The selected appointments are:")
    return sub

In [ ]:
extract_sub(l,max_seq(l))

The longest possible duration is :
110
The selected appointments are:


[20, 50, 40]